# Reranker 설정 (Cohere Rerank 모델)

이 노트북은 검색 결과의 재순위화를 위한 Cohere Rerank 모델 설정 과정을 안내합니다.

In [ ]:
import boto3
import json
from dotenv import load_dotenv

load_dotenv()

## 1. SageMaker 엔드포인트 설정

In [ ]:
# SageMaker 클라이언트 초기화
sagemaker = boto3.client('sagemaker')

# 엔드포인트 설정
endpoint_name = 'Endpoint-Cohere-Rerank-3-Nimble-Model-Multi-1'
model_name = 'cohere-rerank-3-nimble'

# 엔드포인트 생성 설정
endpoint_config = {
    'EndpointConfigName': f'{endpoint_name}-config',
    'ProductionVariants': [{
        'VariantName': 'AllTraffic',
        'ModelName': model_name,
        'InstanceType': 'ml.g5.xlarge',
        'InitialInstanceCount': 1
    }]
}

## 2. API Gateway 설정

In [ ]:
# API Gateway 클라이언트 초기화
apigateway = boto3.client('apigateway')

# API 생성
api_name = 'rerank-api'
api = apigateway.create_rest_api(name=api_name)
api_id = api['id']

# Lambda 함수 설정
with open('manifest/lambda-handler.py', 'r') as file:
    lambda_code = file.read()

# Lambda 함수 생성 및 API Gateway 연동
lambda_client = boto3.client('lambda')
lambda_response = lambda_client.create_function(
    FunctionName='rerank-handler',
    Runtime='python3.8',
    Role='your-lambda-role-arn',  # IAM 역할 ARN 필요
    Handler='lambda_function.lambda_handler',
    Code={'ZipFile': lambda_code.encode()}
)

## 3. 재순위화 테스트

In [ ]:
# 테스트 데이터
test_data = {
    "documents": [
        {"content": "테스트 문서 1"},
        {"content": "테스트 문서 2"}
    ],
    "query": "테스트 쿼리",
    "top_n": 2
}

# API 엔드포인트 테스트
import requests

api_url = 'your-api-endpoint'  # API Gateway URL
response = requests.post(api_url, json=test_data)
print("재순위화 결과:", response.json())